In [1]:
from __future__ import unicode_literals, print_function, division

import random
import re
from io import open
import csv
import unicodedata

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, RandomSampler, DataLoader
from torch import optim
import numpy as np
import matplotlib.pyplot as plt 

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [10]:
S = 0
E = 1

class Lang:
    def __init__(self, name):
        self.name = name
        self.wtoi = {}
        self.wordToCount = {}
        self.itow = {0:'<S>', 1:'<E>'}
        self.nwords = 2
    
    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)
    
    def addWord(self, word):
        if word not in self.wtoi:
            self.wtoi[word] = self.nwords
            self.wordToCount[word] = 1
            self.itow[self.nwords] = word
            self.nwords += 1
        
        else:
            self.wordToCount[word] += 1
    

In [11]:
# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z!?]+", r" ", s)
    return s.strip()

In [12]:
def cleanCsv(inputFile, outputFile):
    with open(inputFile, 'r', encoding='utf-8') as infile, open(outputFile, 'w', encoding='utf-8') as outfile:
        reader = csv.reader(infile, delimiter='\t')
        writer = csv.writer(outfile, delimiter='\t')

        for row in reader:
            if len(row) >= 2:
                writer.writerow(row[:2])
cleanCsv('slk.txt', 'eng-slk.txt')
print('Done....!')

Done....!


In [13]:
def readLang(lang1, lang2, reverse=False):
    lines = open('data/%s-%s.txt' % (lang1, lang2), encoding='utf-8').read().strip().split('\n')

    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    if reverse:
        pairs = [list(reversed(p)) for p in pairs] #reverse all pairs
        inputLang = Lang(lang2)
        outputLang = Lang(lang1)
    
    else:
        inputLang = Lang(lang1)
        outputLang = Lang(lang2)
    
    return inputLang, outputLang, pairs

In [14]:
MAXLEN = 10

engPrefix = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "

)

def filterPair(p):
    return (
        len(p[0].split(' ')) < MAXLEN and len(p[1].split(' ')) < MAXLEN and p[1].startswith(engPrefix)
    )

def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [15]:
def prepData(lang1, lang2, reverse=False):
    inputLang, outputLang, pairs = readLang(lang1, lang2, reverse)

    print('Len pairs_:', len(pairs))
    pairs = filterPairs(pairs)

    print('counting Words..........')
    for pair in pairs:
        inputLang.addSentence(pair[0])
        outputLang.addSentence(pair[1])

    print('\nCounted Words_:')
    print(inputLang.name, inputLang.nwords)
    print(outputLang.name, outputLang.nwords)

    return inputLang, outputLang, pairs

In [16]:
inputLang, outputLang, pairs = prepData('eng', 'fra', True)
print(random.choice(pairs))

Len pairs_: 135842
counting Words..........

Counted Words_:
fra 4601
eng 2991
['elle est partie en voyage', 'she s gone on a trip']


In [17]:
class EncoderRNN(nn.Module):
    def __init__(self, nHidden, nInput, dropout=0.1):
        super(EncoderRNN, self).__init__()
        self.nHidden = nHidden
        
        self.emb = nn.Embedding(nInput, nHidden)
        self.gru = nn.GRU(nHidden, nHidden, batch_first=True)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, input):
        emb = self.dropout(self.emb(input))
        output, hidden = self.gru(emb)

        return output, hidden



In [18]:
class BahdanauAttention(nn.Module):
    def __init__(self, nHidden):
        super(BahdanauAttention, self).__init__()
        self.Wa = nn.Linear(nHidden, nHidden, )
        self.Ua = nn.Linear(nHidden, nHidden)
        self.Va = nn.Linear(nHidden, 1)

    def forward(self, query, keys):
        scores = self.Va(torch.tanh(self.Wa(query) + self.Ua(keys)))
        scores = scores.squeeze(2).unsqueeze(1)

        weights = F.softmax(scores, dim=1)
        contxt = torch.bmm(weights, keys)

        return weights, contxt

class AttentionDecoderRNN(nn.Module):
    def __init__(self, nHidden, nOutput, dropout=0.1):
        super(AttentionDecoderRNN, self).__init__()
        self.emb = nn.Embedding(nOutput, nHidden)
        self.atten = BahdanauAttention(nHidden)
        self.gru = nn.GRU(2 * nHidden, nHidden, batch_first=True)
        self.out = nn.Linear(nHidden, nOutput)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, encOutput, encHidden, targtTensor=None):
        batchSiz = encOutput.size(0)
        decodInput = torch.empty(batchSiz, 1, dtype=torch.long, device=device).fill_(S)
        decodHidden = encHidden
        decodOutputs = []
        attentions = []

        for i in range(MAXLEN):
            decoder_output, decodHidden, attn_weights = self.forward_step(
                decodInput, decodHidden, encOutput
            )
            decodOutputs.append(decoder_output)
            attentions.append(attn_weights)

            if targtTensor is not None:
                # Teacher forcing: Feed the target as the next input
                decodInput = targtTensor[:, i].unsqueeze(1) # Teacher forcing
            else:
                # Without teacher forcing: use its own predictions as the next input
                _, topi = decoder_output.topk(1)
                decodInput = topi.squeeze(-1).detach()  # detach from history as input

        decodOutputs = torch.cat(decodOutputs, dim=1)
        decodOutputs = F.log_softmax(decodOutputs, dim=-1)
        attentions = torch.cat(attentions, dim=1)

        return decodOutputs, decodHidden, attentions


    def forward_step(self, input, hidden, encOutput):
        emb =  self.dropout(self.emb(input))

        query = hidden.permute(1, 0, 2)
        context, attn_weights = self.atten(query, encOutput)
        input_gru = torch.cat((emb, context), dim=2)

        output, hidden = self.gru(input_gru, hidden)
        output = self.out(output)

        return output, hidden, attn_weights

In [19]:
def indexesFromSentence(lang, sentence):
    return [lang.wtoi[word] for word in sentence.split(' ')]

def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(E)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(1, -1)

def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(inputLang, pair[0])
    target_tensor = tensorFromSentence(outputLang, pair[1])
    return (input_tensor, target_tensor)

def get_dataloader(batch_size):
    input_lang, output_lang, pairs = prepData('eng', 'fra', True)

    n = len(pairs)
    input_ids = np.zeros((n, MAXLEN), dtype=np.int32)
    target_ids = np.zeros((n, MAXLEN), dtype=np.int32)

    for idx, (inp, tgt) in enumerate(pairs):
        inp_ids = indexesFromSentence(input_lang, inp)
        tgt_ids = indexesFromSentence(output_lang, tgt)
        
        inp_ids.append(E)
        tgt_ids.append(E)
        
        input_ids[idx, :len(inp_ids)] = inp_ids
        target_ids[idx, :len(tgt_ids)] = tgt_ids

    train_data = TensorDataset(torch.LongTensor(input_ids).to(device),
                               torch.LongTensor(target_ids).to(device))

    train_sampler = RandomSampler(train_data)
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
    return input_lang, output_lang, train_dataloader

In [20]:
def train_epoch(dataloader, enc, decod, encOptim, decodOptim, criterion):
    total_loss = 0
    for data in dataloader:
        inputTensor, targetTensor = data

        # Zero gradients
        encOptim.zero_grad()
        decodOptim.zero_grad()

        # Forward pass
        encoder_outputs, encoder_hidden = enc(inputTensor)
        decoder_outputs, _, _ = decod(encoder_outputs, encoder_hidden, targetTensor)

        # Compute loss
        loss = criterion(
            decoder_outputs.view(-1, decoder_outputs.size(-1)),
            targetTensor.view(-1)
        )

        # Backward pass and optimization
        loss.backward()
        encOptim.step()
        decodOptim.step()

        total_loss += loss.item()

    return total_loss / len(dataloader)

def train(train_dataloader, encoder, decoder, n_epochs, learning_rate=0.001):
    encOptim = optim.Adam(encoder.parameters(), lr=learning_rate)
    decodOptim = optim.Adam(decoder.parameters(), lr=learning_rate)
    criterion = nn.NLLLoss()

    lossi = []  # Store loss for each epoch

    for i in range(n_epochs):
        loss = train_epoch(train_dataloader, encoder, decoder, encOptim, decodOptim, criterion)
        lossi.append(loss)

    return lossi  # Return all epoch losses for plotting


In [21]:
def evaluate(enc, decod, sentenc, inputLang, outputLang):
    with torch.no_grad():
        inputTensor = tensorFromSentence(inputLang, sentenc)

        encOutput, encHidde = enc(inputTensor)
        decodOutputs, decodHidden, decoder_attn = decod(encOutput, encHidde)

        _, topi = decodOutputs.topk(1)
        decoded_ids = topi.squeeze()

        decoded_words = []
        for idx in decoded_ids:
            if idx.item() == E:
                decoded_words.append('<EOS>')
                break
            decoded_words.append(outputLang.itow[idx.item()])
    return decoded_words, decoder_attn

In [22]:
def evaluateRandomly(enc, decod, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        outputWords, _ = evaluate(enc, decod, pair[0], inputLang, outputLang)
        output_sentence = ' '.join(outputWords)
        print('<', output_sentence)
        print('')

In [24]:
hiddenSiz = 128
batchSiz = 32

inputLang, outputLAng, trainDataloder = get_dataloader(batchSiz)

encoder = EncoderRNN(inputLang.nwords, hiddenSiz).to(device)
decoder = AttentionDecoderRNN(hiddenSiz, outputLAng.nwords).to(device)

train(trainDataloder, encoder, decoder, 8)

Len pairs_: 135842
counting Words..........

Counted Words_:
fra 4601
eng 2991


/home/dkcode/miniconda3/envs/tfenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


IndexError: index out of range in self